In [1]:
import pandas as pd
import numpy as np
from collections import Counter

import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

## Data Set Information
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.

---

## Attribute Information
- **InvoiceNo**: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'C', it indicates a cancellation.
- **StockCode**: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
- **Description**: Product (item) name. Nominal.
- **Quantity**: The quantity of each product (item) per transaction. Numeric.
- **InvoiceDate**: Invoice date and time. Numeric, representing the day and time when each transaction was generated.
- **UnitPrice**: Unit price. Numeric, representing the product price per unit in sterling.
- **CustomerID**: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
- **Country**: Country name. Nominal, representing the name of the country where each customer resides.


In [2]:
raw_data = pd.read_csv('../data/retail.csv', encoding='ISO-8859-1')
display(raw_data.head(10))

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/10 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/10 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/10 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/10 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/10 8:34,1.69,13047.0,United Kingdom


In [25]:
raw_product_descriptions = pd.read_csv('../data/main_product_descriptions.csv')
display(raw_product_descriptions.head(10))

,item_family,item_family_description,category
0,10002,INFLATABLE POLITICAL GLOBE,GIFT ITEMS
1,10080,GROOVY CACTUS INFLATABLE,TOYS
2,10120,DOGGY RUBBER,TOYS
3,10123,HEARTS WRAPPING TAPE,STATIONERY
4,10124,BOOKCOVER TAPE,STATIONERY
5,10125,MINI FUNKY DESIGN TAPES,CRAFTS
6,10133,COLOURING PENCILS BROWN TUBE,STATIONERY
7,10135,COLOURING PENCILS BROWN TUBE,STATIONERY
8,11001,ASSTD DESIGN RACING CAR PEN,STATIONERY
9,15030,FAN BLACK FRAME,LIGHTING


### Item Family

In [26]:
item_family = raw_product_descriptions.copy()
item_family = item_family.rename(columns={
    'item_family': 'item_family_id',
    'category': 'item_category'
})

item_family['item_family_id'] = item_family['item_family_id'].astype(int)
item_family['item_family_description'] = item_family['item_family_description'].astype(str)
item_family['item_category'] = item_family['item_category'].astype(str)
display(item_family.head(10))

,item_family_id,item_family_description,item_category
0,10002,INFLATABLE POLITICAL GLOBE,GIFT ITEMS
1,10080,GROOVY CACTUS INFLATABLE,TOYS
2,10120,DOGGY RUBBER,TOYS
3,10123,HEARTS WRAPPING TAPE,STATIONERY
4,10124,BOOKCOVER TAPE,STATIONERY
5,10125,MINI FUNKY DESIGN TAPES,CRAFTS
6,10133,COLOURING PENCILS BROWN TUBE,STATIONERY
7,10135,COLOURING PENCILS BROWN TUBE,STATIONERY
8,11001,ASSTD DESIGN RACING CAR PEN,STATIONERY
9,15030,FAN BLACK FRAME,LIGHTING


### Transactions

In [24]:
transactions = raw_data.copy()

transactions = transactions.rename(columns={
    'InvoiceNo': 'invoice_id',
    'StockCode': 'item_id',
    'Description': 'item_description',
    'Quantity': 'quantity_amount',
    'InvoiceDate': 'event_timestamp_invoiced_at',
    'UnitPrice': 'unit_price_eur',
    'CustomerID': 'customer_id',
    'Country': 'country_name'
})

transactions['invoice_id'] = transactions['invoice_id'].astype(str)
transactions['item_id'] = transactions['item_id'].astype(str)
transactions['item_description'] = transactions['item_description'].astype(str)
transactions['quantity_amount'] = transactions['quantity_amount'].astype(int)
transactions['event_timestamp_invoiced_at'] = pd.to_datetime(transactions['event_timestamp_invoiced_at'])
transactions['unit_price_eur'] = transactions['unit_price_eur'].astype(float)
transactions['customer_id'] = transactions['customer_id'].astype('Int64')
transactions['country_name'] = transactions['country_name'].astype(str)
transactions.index.name = 'transaction_id'
transactions.reset_index(inplace=True)
transactions['transaction_id'] = transactions['transaction_id'].astype('Int64')

transactions['is_return'] = transactions['quantity_amount'] < 0

transactions.head(10)

/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_74621/49388509.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transactions['event_timestamp_invoiced_at'] = pd.to_datetime(transactions['event_timestamp_invoiced_at'])


,transaction_id,invoice_id,item_id,item_description,quantity_amount,event_timestamp_invoiced_at,unit_price_eur,customer_id,country_name,is_return
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,False
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,False
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,False
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,False
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,False
5,5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,False
6,6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom,False
7,7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,False
8,8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,False
9,9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,False


### Items

In [5]:
items = transactions.groupby(['item_id', 'item_description']).count()[[]].reset_index()
items.sample(20)

,item_id,item_description
1908,22405,MONEY BOX POCKET MONEY DESIGN
3299,23454,THREE MINI HANGING FRAMES
4826,84944,nan
1147,21688,mixed up
4641,84796B,nan
2883,23131,?
45,16008,check
2674,22985,"WRAP, BILLBOARD FONTS DESIGN"
2925,23159,nan
5003,85045,nan


In [6]:
items['is_operational_item'] = (items['item_id'].str.len() < 5) | (items['item_id'].str.contains('gift', case=False))
items['item_family_id'] = items['item_id'].str.extract(r'^(\d+)', expand=False)
items['item_variant'] = items['item_id'].str.extract(r'(\D+)$', expand=False).fillna('')
items['item_variant'] = np.where(items['item_id'] == items['item_family_id'],  np.nan, items['item_variant'])

items.head()

,item_id,item_description,is_operational_item,item_family_id,item_variant
0,10002,INFLATABLE POLITICAL GLOBE,False,10002,NaN
1,10002,nan,False,10002,NaN
2,10080,GROOVY CACTUS INFLATABLE,False,10080,NaN
3,10080,check,False,10080,NaN
4,10080,nan,False,10080,NaN


#### Set items additional info flags

In [7]:
selected_items = items[items['is_operational_item'] == False]

selected_items['is_all_uppercase'] = selected_items['item_description'].str.isupper()
selected_items['description_len'] = selected_items['item_description'].str.len()
selected_items = selected_items.sort_values(by=['item_id', 'description_len'], ascending=[True, False])
selected_items['description_order'] = selected_items.groupby('item_id').cumcount() + 1

selected_items.head()

/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_74621/3455374893.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['is_all_uppercase'] = selected_items['item_description'].str.isupper()
/var/folders/hv/ckh3m6gn1sd45q2qctrqcwzh0000gn/T/ipykernel_74621/3455374893.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_items['description_len'] = selected_items['item_description'].str.len()


,item_id,item_description,is_operational_item,item_family_id,item_variant,is_all_uppercase,description_len,description_order
0,10002,INFLATABLE POLITICAL GLOBE,False,10002,NaN,True,27,1
1,10002,nan,False,10002,NaN,False,3,2
2,10080,GROOVY CACTUS INFLATABLE,False,10080,NaN,True,24,1
3,10080,check,False,10080,NaN,False,5,2
4,10080,nan,False,10080,NaN,False,3,3


In [8]:
weird_items = selected_items[((selected_items['description_order'] > 1) & (selected_items['description_len'] < 13)) | ~selected_items['is_all_uppercase']]
description_list = weird_items['item_description'].drop_duplicates().tolist()
print(description_list)

is_unknown_item = []
is_error_item = []
is_special_item = []
is_modification_item = []

for description in description_list:
    description_lower = description.lower()
    if any(word in description_lower for word in [
        'check', 'damage', 'wet', 'lost', 'found', 'error', 'wrong', 'faulty', 'mouldy', 'smashed', 'broken', 'crush', 'crack', 'unsaleable', 'missing', 'throw away'
    ]):
        is_error_item.append(description)
    elif any(word in description_lower for word in [
        'dotcom', 'amazon', 'fba', 'ebay', 'john lewis', 'showroom', 'voucher', 'cordial jug', 'website'
    ]):
        is_special_item.append(description)
    elif any(word in description_lower for word in [
        'adjust', 'sample', 'sold as', 'mailout', 'allocate', 'temp', 'credit', 'sale', 're-adjustment', 'label mix up'
    ]):
        is_modification_item.append(description)


is_unknown_item = ['?','??','???','Incorrect stock entry.',"can't find",'nan',None]

is_error_item += [
    'on cargo order', 'test', 'barcode problem', 'mixed up', 'michel oops', 'printing smudges/thrown away', 
    'had been put aside', 'rusty thrown away', 'incorrectly made-thrown away.', 'Breakages', 'counted', 
    'Had been put aside.', 'returned', 'thrown away', 'mystery! Only ever imported 1800', 'Dagamed', 
    'code mix up? 84930', 'Printing smudges/thrown away', 'came coded as 20713', 'incorrect stock entry.',
    "thrown away-can't sell",'Thrown away-rusty','Thrown away.','Given away','historic computer difference?....se',
    'alan hodge cant mamage this section',"thrown away-can't sell.", 'label mix up','sold in set?','mix up with c'
]

is_special_item += [
    'MIA', '?display?', 'Amazon Adjustment', 'Lighthouse Trading zero invc incorr', 
    'Dotcomgiftshop Gift Voucher £100.00', 'sold as set?', 
    'High Resolution Image', 'John Lewis','Bank Charges','Next Day Carriage'
]

is_modification_item += [
    'Adjustment', 'OOPS ! adjustment', 'reverse 21/5/10 adjustment', 'reverse previous adjustment', 
    'marked as 23343', 'incorrectly put back into stock', 'Not rcvd in 10/11/2010 delivery', 'Display', 
    'Had been put aside.',  'sold as set by dotcom', 'add stock to allocate online orders', 
    'allocate stock for dotcom orders ta', 'for online retail orders', 'Marked as 23343'
]

unclassified_items = set(description_list) - set(is_error_item) - set(is_special_item) - set(is_modification_item)

print("is_error_item:", is_error_item)
print("is_special_item:", is_special_item)
print("is_modification_item:", is_modification_item)
print("Unclassified items:", unclassified_items)


['nan', 'check', 'damaged', 'wet/rusty', 'FLOWERS HANDBAG blue and orange', 'alan hodge cant mamage this section', 'THE KING GIFT BAG 25x24x12cm', 'found', 'Adjustment', 'ESSENTIAL BALM 3.5g TIN IN ENVELOPE', 'adjustment', 'dotcom', 'MIA', '?', 'lost in space', 'wrongly marked. 23343 in box', 'wrongly marked 23343', 'wrongly coded 23343', 'wrongly coded-23343', 'Marked as 23343', 'Found', 'damages', '??', 'damages/display', '*USB Office Mirror Ball', 'POLYESTER FILLER PAD 60x40cm', 'Amazon Adjustment', 'taig adjust no stock', 'WET/MOULDY', 'had been put aside', 'damages?', 'Damaged', 'wet', '*Boombox Ipod Classic', 'sold as set on dotcom', 'wet rusty', 'on cargo order', 'wrongly marked', '???', 'broken', 'Show Samples', 'Sold as 1 on dotcom', "Dr. Jam's Arouzer Stress Ball", "Dad's Cab Electronic Meter", 'CHECK', 'mystery! Only ever imported 1800', 'rcvd be air temp fix for dotcom sit', 're dotcom quick fix.', 'samples', '?display?', 'mixed up', 'wrong barcode', 'sold as set on dotcom 

In [22]:
items['is_unknown_item'] = items['item_description'].str.lower().isin(is_unknown_item)
items['is_error_item'] = items['item_description'].str.lower().isin(is_error_item)
items['is_special_item'] = items['item_description'].str.lower().isin(is_special_item)
items['is_modification_item'] = items['item_description'].str.lower().isin(is_modification_item)

items = items[['item_id', 'item_family_id', 'item_description', 'item_variant', 'is_operational_item', 'is_unknown_item', 'is_special_item', 'is_modification_item', 'is_error_item']]

items.loc[items['is_operational_item'], ['item_variant', 'is_unknown_item', 'is_special_item', 'is_modification_item', 'is_error_item']] = [None, False, False, False, False]
items.loc[items['is_unknown_item']] = items.loc[items['is_unknown_item']].replace('', None)
items.loc[items['is_special_item']] = items.loc[items['is_special_item']].replace('', None)
items.loc[items['is_modification_item']] = items.loc[items['is_modification_item']].replace('', None)
items.loc[items['is_error_item']] = items.loc[items['is_error_item']].replace('', None)

items.head(10)

,item_id,item_family_id,item_description,item_variant,is_operational_item,is_unknown_item,is_special_item,is_modification_item,is_error_item
0,10002,10002,INFLATABLE POLITICAL GLOBE,NaN,False,False,False,False,False
1,10002,10002,nan,NaN,False,True,False,False,False
2,10080,10080,GROOVY CACTUS INFLATABLE,NaN,False,False,False,False,False
3,10080,10080,check,NaN,False,False,False,False,True
4,10080,10080,nan,NaN,False,True,False,False,False
5,10120,10120,DOGGY RUBBER,NaN,False,False,False,False,False
6,10123C,10123,HEARTS WRAPPING TAPE,C,False,False,False,False,False
7,10123C,10123,nan,C,False,True,False,False,False
8,10123G,10123,nan,G,False,True,False,False,False
9,10124A,10124,SPOTS ON RED BOOKCOVER TAPE,A,False,False,False,False,False


### Set categories with Open AI LLM

#### Set item family with categories

In [ ]:
filtered_items = items[~items['is_operational_item'] & ~items['is_error_item'] & ~items['is_special_item'] & ~items['is_modification_item']& ~items['is_unknown_item']]
grouped = filtered_items.groupby('item_family_id')['item_description'].apply(lambda x: ' '.join(x))

def get_most_common_words(text):
    words = text.split()  
    word_counts = Counter(words)
    max_count = max(word_counts.values(), default=0)
    
    most_common_words = [word for word, count in word_counts.items() if count == max_count]
    return ' '.join(most_common_words) if most_common_words else None

main_product_descriptions = grouped.apply(get_most_common_words).reset_index()
main_product_descriptions.columns = ['item_family', 'item_family_description']


main_product_descriptions.head(10)

In [ ]:
categories = [
    "HOME DECOR",
    "KITCHENWARE",
    "TABLEWARE",
    "GIFT ITEMS",
    "FURNITURE",
    "TOYS",
    "CANDLES",
    "JEWELRY",
    "TEXTILES",
    "GARDENING",
    "BATHROOM",
    "STATIONERY",
    "BAGS",
    "LIGHTING",
    "CRAFTS",
    "HOLIDAY",
    "OUTDOOR",
    "TOOL & DIY",
    "SPORTS",
    "STORAGE",
    "TRAVEL",
    "BEAUTY",
    "BOOKS",
    "GAMES",
    "ORGANIZERS",
    "TABLE LINEN",
    "CHILDREN",
    "CERAMICS",
    "CLOTHING",
    "PLANTS & FLOWERS",
    "OTHERS"
]

#### LLM extraction

In [ ]:
from openai import OpenAI

import openai_setup

# Create GPT Client
def initChatGPTClient():
    organization = openai_setup.conf['organization']
    project = openai_setup.conf['project']
    key = openai_setup.conf['key']

    client = OpenAI(
        api_key=key,
        organization=organization,
        project=project
    )
    return client

def llm_extract_category(client, item):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert classifier system that gives a category to each object from a commerce. Ansewer only with the category selected in uppercase. The possible categories:" + str(categories)},
            {
                "role": "user",
                "content": str(item)
            }
        ]
    )
    answer = completion.choices[0].message.content
    return answer

In [ ]:
client = initChatGPTClient()
main_product_descriptions['category'] = main_product_descriptions['item_family_description'].apply(lambda item: llm_extract_category(client, item))
display(main_product_descriptions)

In [ ]:
display(main_product_descriptions)

In [ ]:
main_product_descriptions.to_csv('../data/main_product_descriptions.csv', index=False)

### Dash ideas

- beneficio mes
- beneficio año
- mom
- yoy

- numero de compras
- numero de clientes
- beneficio total
- numero de invoices realizados
- numero de objetos comprados totals
- devoluciones
- flags especiales

- por pais

- por categoria de producto

- tipos de clientes:
    - gasto
    - numero de invoices realizados
    - numero de objetos comprados totals
    - numero de objetos por compra
    - numero de compras por mes
    - numero de compras por año
    - devoluciones
    - flags especiales
    - categoria que mas compra